In [ ]:
/storage/ice1/shared/d-pace_community/makerspace-datasets/MEDICAL/OLIVES

In [26]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from torch.autograd import Variable

data = np.genfromtxt('/content/Biomarker_Clinical_Data_Images.csv', delimiter=',', skip_header=1, usecols=(2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20))

with_headers = np.genfromtxt('/content/Biomarker_Clinical_Data_Images.csv', delimiter=',', names=True, usecols=(2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20))

avgArr = np.zeros([int(data.shape[0]/49), data.shape[1]])

for i in range(0, int(data.shape[0]/49)):
  start = 49*i+1
  scans = data[start:start+48,:]
  avgArr[i] = scans.mean(axis=0)

np.set_printoptions(precision=3, suppress=True)

presence = np.ceil(avgArr)

biomarkerAvg, temp = np.hsplit(avgArr, np.array([16]))
__, scores = np.hsplit(temp, np.array([1]))
biomarkerPresence, __ = np.hsplit(presence, np.array([16]))

avg_train, avg_test, score_train, score_test = train_test_split(biomarkerAvg, scores, train_size = 150, test_size=42, random_state=0)
presence_train, presence_test, __, __ = train_test_split(biomarkerPresence, scores, train_size = 150, test_size=42, random_state=0)

score_train = (score_train - score_train.min(0))/score_train.ptp(0)
score_test = (score_test - score_test.min(0))/score_test.ptp(0)



In [51]:
torch_avg_train = Variable(torch.from_numpy(avg_train).float())
torch_avg_test = Variable(torch.from_numpy(avg_test).float())
torch_presence_train = Variable(torch.from_numpy(presence_train).float())
torch_presence_test = Variable(torch.from_numpy(presence_test).float())
torch_score_train = Variable(torch.from_numpy(score_train).float())
torch_score_test = Variable(torch.from_numpy(score_test).float())

class FullLinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(FullLinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(2, 4)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(4, 16)
    def forward(self, x):
        X = self.linear(x)
        X2 = self.relu(X)
        y_pred = self.linear2(X2)
        return y_pred

In [49]:
class SingleLinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(SingleLinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(2, 1)
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [ ]:
del model
model = FullLinearRegressionModel()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
for epoch in range(5000):

    # Forward pass: Compute predicted y by passing
    # x to the model
    pred_y = model(torch_score_train)

    # Compute and print loss
    loss = loss_fn(pred_y, torch_presence_train)

    # Zero gradients, perform a backward pass,
    # and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch % 20 == 0):
      print('epoch {}, loss {}'.format(epoch, loss.item()))

test_pred = model(torch_score_test)
np.set_printoptions(precision=3, suppress=True)
torch.set_printoptions(threshold=10_000)
print(test_pred)

In [66]:
for i in range(0,16):
  del singleModel
  print(with_headers.dtype.names[i])
  biomarker = biomarkerPresence[:,i].reshape(-1,1)

  bio_train, bio_test, __, __ = train_test_split(biomarker, scores, train_size = 150, test_size=42, random_state=0)

  torch_bio_train = Variable(torch.from_numpy(bio_train).float())
  torch_bio_test = Variable(torch.from_numpy(bio_test).float())

  singleModel = SingleLinearRegressionModel()
  loss_fn = torch.nn.MSELoss()
  optimizer = torch.optim.SGD(singleModel.parameters(), lr = 0.01)
  for epoch in range(100):
    pred_y = singleModel(torch_score_train)
    loss = loss_fn(pred_y, torch_bio_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #if (epoch % 50 == 0):
    #  print('epoch {}, loss {}'.format(epoch, loss.item()))
  pred = singleModel(torch_score_test)
  loss = loss_fn(pred, torch_bio_test)
  print(loss)

Atrophy__thinning_of_retinal_layers
tensor(0.0915, grad_fn=<MseLossBackward0>)
Disruption_of_EZ
tensor(0.2100, grad_fn=<MseLossBackward0>)
DRIL
tensor(0.0253, grad_fn=<MseLossBackward0>)
IR_hemorrhages
tensor(0.3031, grad_fn=<MseLossBackward0>)
IR_HRF
tensor(0.0301, grad_fn=<MseLossBackward0>)
Partially_attached_vitreous_face
tensor(0.2425, grad_fn=<MseLossBackward0>)
Fully_attached_vitreous_face
tensor(0.2838, grad_fn=<MseLossBackward0>)
Preretinal_tissuehemorrhage
tensor(0.1907, grad_fn=<MseLossBackward0>)
Vitreous_debris
tensor(0.0731, grad_fn=<MseLossBackward0>)
VMT
tensor(0.0675, grad_fn=<MseLossBackward0>)
DRTME
tensor(0.3008, grad_fn=<MseLossBackward0>)
Fluid_IRF
tensor(0.1758, grad_fn=<MseLossBackward0>)
Fluid_SRF
tensor(0.1012, grad_fn=<MseLossBackward0>)
Disruption_of_RPE
tensor(0.0323, grad_fn=<MseLossBackward0>)
PED_serous
tensor(0.0040, grad_fn=<MseLossBackward0>)
SHRM
tensor(0.1430, grad_fn=<MseLossBackward0>)
